In [1]:
import warnings

import pandas as pd
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

In [2]:
import scipy.stats as stats
import numpy as np


class BlackScholes:
    """
    This is a class for Options contract for pricing European options on stocks/index without dividends.

    Attributes:
        spot          : int or float
        strike        : int or float
        rate          : float
        days_to_expiry: int or float [days to expiration in number of years]
        volatility    : float
        call_price     : int or float [default None]
        put_price      : int or float [default None]
    """

    def __init__(
        self,
        spot,
        strike,
        rate,
        days_to_expiry,
        volatility,
        dividend=None,
        call_price=None,
        put_price=None,
    ):
        # Spot Price
        self.spot = spot

        # Option Strike
        self.strike = strike

        # Interest Rate
        self.rate = rate

        # Dividend Rate
        self.dividend = dividend

        # Days To Expiration
        self.days_to_expiry = days_to_expiry

        # Volatility
        self.volatility = volatility

        # Call price # mkt price
        self.call_price = call_price

        # Put price # mkt price
        self.put_price = put_price

        # Utility
        self._a_ = self.volatility * self.days_to_expiry**0.5

        if self.strike == 0:
            raise ZeroDivisionError("The strike price cannot be zero")
        else:
            if self.dividend is None:
                self._d1_ = (
                    np.log(self.spot / self.strike)
                    + (self.rate + (self.volatility**2) / 2) *
                    self.days_to_expiry
                ) / self._a_
            else:
                self._d1_ = (
                    np.log(self.spot / self.strike)
                    + (self.rate - self.dividend + (self.volatility**2) / 2)
                    * self.days_to_expiry
                ) / self._a_

        self._d2_ = self._d1_ - self._a_

        self._b_ = np.e ** -(self.rate * self.days_to_expiry)
        """
        Contains all the attributes defined for the object itself. It maps the attribute name to its value.
        """
        for i in [
            "call_price",
            "put_price",
            "call_delta",
            "put_delta",
            "call_theta",
            "put_theta",
            "call_rho",
            "put_rho",
            "vega",
            "gamma",
            "implied_vol",
        ]:
            self.__dict__[i] = None

        [self.call_price, self.put_price] = self._price()
        [self.call_delta, self.put_delta] = self._delta()
        [self.call_theta, self.put_theta] = self._theta()
        [self.call_rho, self.put_rho] = self._rho()
        self.vega = self._vega()
        self.gamma = self._gamma()

    # Option Price
    def _price(self):
        """Returns the option price: [Call price, Put price]"""

        if self.volatility == 0 or self.days_to_expiry == 0:
            call = np.maximum(0.0, self.spot - self.strike)
            put = np.maximum(0.0, self.strike - self.spot)
        else:
            call = self.spot * stats.norm.cdf(self._d1_) - self.strike * np.e ** (
                -self.rate * self.days_to_expiry
            ) * stats.norm.cdf(self._d2_)

            put = self.strike * np.e ** (-self.rate * self.days_to_expiry) * stats.norm.cdf(
                -self._d2_
            ) - self.spot * stats.norm.cdf(-self._d1_)
        return [call, put]

    # Option Delta
    def _delta(self):
        """Returns the option delta: [Call delta, Put delta]"""
        call = stats.norm.cdf(self._d1_)
        put = -stats.norm.cdf(-self._d1_)
        return [call, put]

    # Option Gamma
    def _gamma(self):
        """Returns the option gamma"""
        return stats.norm.pdf(self._d1_) / (self.spot * self._a_)

    # Option Vega
    def _vega(self):
        """Returns the option vega"""
        if self.volatility == 0 or self.days_to_expiry == 0:
            return 0.0
        else:
            return self.spot * stats.norm.pdf(self._d1_) * self.days_to_expiry**0.5 / 100

    # Option Theta
    def _theta(self):
        """Returns the option theta: [Call theta, Put theta]"""
        call = -self.spot * stats.norm.pdf(self._d1_) * self.volatility / (
            2 * self.days_to_expiry**0.5
        ) - self.rate * self.strike * self._b_ * stats.norm.cdf(self._d2_)

        put = -self.spot * stats.norm.pdf(self._d1_) * self.volatility / (
            2 * self.days_to_expiry**0.5
        ) + self.rate * self.strike * self._b_ * stats.norm.cdf(-self._d2_)
        return [call / 365, put / 365]

    # Option Rho
    def _rho(self):
        """Returns the option rho: [Call rho, Put rho]"""
        call = self.strike * self.days_to_expiry * \
            self._b_ * stats.norm.cdf(self._d2_) / 100
        put = -self.strike * self.days_to_expiry * \
            self._b_ * stats.norm.cdf(-self._d2_) / 100
        return [call, put]

In [3]:
class BinomialMethod:
    """
    This is a class for Options contract for pricing European options on stocks/index without dividends.
    Attributes:
        spot          : int or float
        strike        : int or float
        rate          : float
        volatility    : float
        big_t         : int or float [days to expiration in number of years]
        n_steps       : int [number of steps]
    """

    def __init__(
        self,
        spot,
        strike,
        rate,
        volatility,
        time_to_expiry,
        n_steps,
        call_price=None,
        put_price=None,
    ):
        # Spot Price
        self.spot = spot

        # Option Strike
        self.strike = strike

        # Interest Rate
        self.rate = rate

        # Volatility
        self.volatility = volatility

        # Days To Expiration
        self.time_to_expiry = time_to_expiry

        # Number of Steps
        self.n_steps = n_steps

        # Call price
        self.call_price = call_price

        # Put price
        self.put_price = put_price

        # Utility

        self._dt_ = self.time_to_expiry / self.n_steps
        self._u_ = np.exp(self.volatility * np.sqrt(self._dt_))
        self._d_ = 1 / self._u_
        self._p_ = (np.exp(self.rate * self._dt_) -
                    self._d_) / (self._u_ - self._d_)
        self._d1_ = (
            np.log(self.spot / self.strike)
            + (self.rate + 0.5 * self.volatility**2) * self.time_to_expiry
        ) / (self.volatility * np.sqrt(self.time_to_expiry))
        self._d2_ = self._d1_ - self.volatility * np.sqrt(self.time_to_expiry)

        if self.strike == 0:
            raise ZeroDivisionError("The strike price cannot be zero")
        else:
            pass

        for i in [
            "call_price",
            "put_price",
            "call_delta",
            "put_delta",
            "gamma",
            "vega",
            "call_theta",
            "put_theta",
        ]:
            self.__dict__[i] = None

        [self.call_price, self.put_price] = self._price()
        [self.call_delta, self.put_delta] = self._delta()
        self.gamma = self._gamma()
        self.vega = self._vega()
        [self.call_theta, self.put_theta] = self._theta()

    def _price(self):
        """
        This function calculates the price of the option using the binomial method.
        """
        # Initialize the stock price tree
        stock_price = np.zeros((self.n_steps + 1, self.n_steps + 1))
        stock_price[0, 0] = self.spot
        for i in range(1, self.n_steps + 1):
            stock_price[i, 0] = stock_price[i - 1, 0] * self._u_
            for j in range(1, i + 1):
                stock_price[i, j] = stock_price[i - 1, j - 1] * self._d_

        # Initialize the option value tree
        call = np.zeros((self.n_steps + 1, self.n_steps + 1))
        put = np.zeros((self.n_steps + 1, self.n_steps + 1))
        for j in range(self.n_steps + 1):
            call[self.n_steps, j] = max(
                stock_price[self.n_steps, j] - self.strike, 0)
            put[self.n_steps, j] = max(
                self.strike - stock_price[self.n_steps, j], 0)

        # Calculate the option value at each node
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                call[i, j] = np.exp(-self.rate * self._dt_) * (
                    self._p_ * call[i + 1, j] +
                    (1 - self._p_) * call[i + 1, j + 1]
                )
                put[i, j] = np.exp(-self.rate * self._dt_) * (
                    self._p_ * put[i + 1, j] +
                    (1 - self._p_) * put[i + 1, j + 1]
                )
        return [call[0, 0], put[0, 0]]

    def _delta(self):
        call_delta = stats.norm.cdf(self._d1_)
        put_delta = stats.norm.cdf(self._d1_) - 1
        return [call_delta, put_delta]

    def _gamma(self):
        gamma = stats.norm.pdf(self._d1_) / (
            self.spot * self.volatility * np.sqrt(self.time_to_expiry)
        )
        return gamma

    def _vega(self):
        vega = self.spot * \
            stats.norm.pdf(self._d1_) * np.sqrt(self.time_to_expiry)
        return vega

    def _theta(self):
        call_theta = -(self.spot * stats.norm.pdf(self._d1_) * self.volatility) / (
            2 * np.sqrt(self.time_to_expiry)
        ) - self.rate * self.strike * np.exp(-self.rate * self.time_to_expiry) * stats.norm.cdf(
            self._d2_
        )
        put_theta = -(self.spot * stats.norm.pdf(self._d1_) * self.volatility) / (
            2 * np.sqrt(self.time_to_expiry)
        ) + self.rate * self.strike * np.exp(-self.rate * self.time_to_expiry) * stats.norm.cdf(
            -self._d2_
        )
        return [call_theta, put_theta]

In [4]:
from scipy.integrate import quad
# Heston characteristic function


def heston_charfunc(
    phi,
    initial_stock_price,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    # constants
    a = kappa * theta
    b = kappa + lambd
    # common terms w.r.t phi
    rspi = rho * sigma * phi * 1j
    # define d parameter given phi and b
    d = np.sqrt((rho * sigma * phi * 1j - b) **
                2 + (phi * 1j + phi**2) * sigma**2)
    # define g parameter given phi, b and d
    g = (b - rspi + d) / (b - rspi - d)
    # calculate characteristic function by components
    exp1 = np.exp(risk_free_rate * phi * 1j * tau)
    term2 = initial_stock_price ** (phi * 1j) * (
        (1 - g * np.exp(d * tau)) / (1 - g)
    ) ** (-2 * a / sigma**2)
    exp2 = np.exp(
        a * tau * (b - rspi + d) / sigma**2
        + initial_variance
        * (b - rspi + d)
        * ((1 - np.exp(d * tau)) / (1 - g * np.exp(d * tau)))
        / sigma**2
    )
    return exp1 * term2 * exp2


# Heston integrand
def integrand(
    phi,
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )
    numerator = np.exp(risk_free_rate * tau) * heston_charfunc(
        phi - 1j, *args
    ) - strike * heston_charfunc(phi, *args)
    denominator = 1j * phi * strike ** (1j * phi)
    return numerator / denominator


# Heston price using rectangular integration
def heston_price_rec(
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )

    P, umax, N = 0, 100, 10000
    dphi = umax / N  # dphi is width

    for i in range(1, N):
        # rectangular integration
        phi = dphi * (2 * i + 1) / 2  # midpoint to calculate height
        numerator = np.exp(risk_free_rate * tau) * heston_charfunc(
            phi - 1j, *args
        ) - strike * heston_charfunc(phi, *args)
        denominator = 1j * phi * strike ** (1j * phi)
        P += dphi * numerator / denominator
    call = np.real(
        (initial_stock_price - strike * np.exp(-risk_free_rate * tau)) / 2 + P / np.pi
    )
    put = call - initial_stock_price + strike * np.exp(-risk_free_rate * tau)
    return [call, put]


# Heston price using quadrature integration
def heston_price(
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        strike,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )

    real_integral, err = np.real(quad(integrand, 0, 100, args=args))
    call = (
        initial_stock_price - strike * np.exp(-risk_free_rate * tau)
    ) / 2 + real_integral / np.pi
    put = call - initial_stock_price + strike * np.exp(-risk_free_rate * tau)
    return [call, put, err]

In [5]:
# Parameters to test model

S0 = 100.  # initial asset price
K = 100.  # strike
v0 = 0.1  # initial variance
r = 0.03  # risk free rate
kappa = 1.5768  # rate of mean reversion of variance process
theta = 0.0398  # long-term mean variance
sigma = 0.3  # volatility of volatility
lambd = 0.575  # risk premium of variance
rho = -0.5711  # correlation between variance and stock process
tau = 1.  # time to maturity

In [6]:
heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

[11.540361819355368, 8.584915174206188, 4.461749063147755e-09]

In [7]:
heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

[11.521145092601861, 8.565698447452675]

In [8]:
from yahoo_fin import options

# Get the options chain for a given ticker
ticker = "^SPX"
spot = 4517.33
raw_expiration_date = options.get_expiration_dates(ticker)

In [9]:
raw_expiration_date[:-39]

['November 16, 2023',
 'November 17, 2023',
 'November 20, 2023',
 'November 21, 2023',
 'November 22, 2023',
 'November 24, 2023',
 'November 27, 2023',
 'November 28, 2023',
 'November 29, 2023',
 'November 30, 2023',
 'December 1, 2023',
 'December 4, 2023',
 'December 5, 2023']

In [10]:
expiration_dates = raw_expiration_date[:-39]
# df_chain = options.get_options_chain(ticker, expiration_dates[1])
# calls = df_chain['calls']

In [11]:
# filtered_calls = calls[(calls['Strike'] >= 4350) & (calls['Strike'] <= 4500)]
# filtered_calls.reset_index(drop=True, inplace=True)
# filtered_calls

In [12]:
# new_df = filtered_calls[['Strike', 'Last Price','Implied Volatility']]
# new_df['Expiration_Date'] = expiration_dates[1]
# new_df.set_index('Expiration_Date', inplace=True)

In [13]:
chain_dict = {}
# Loop through the expiration data and get the options chain for each date and store it in a dictionary
for dates in expiration_dates:
    chain_dict[dates] = options.get_options_chain(ticker, dates)

df_list = []

# Loop through the dictionary and append each dataframe to the list
for key in chain_dict:
    df_list.append(chain_dict[key]['calls'])

# Use pandas.concat() to concatenate all dataframes into one
merged_df = pd.concat(df_list)

In [14]:
# Filter the dataframe for the desired strike prices (centered Strikes)
filtered_calls_chain = merged_df[(merged_df['Strike'] >= 4400) & (merged_df['Strike'] <= 4550)]
filtered_calls_chain.reset_index(drop=True, inplace=True)
filtered_calls_chain

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,SPXW231116C04400000,2023-11-15 4:13PM EST,4400.0,99.46,100.8,102.0,0.0,-,100,0,0.00%
1,SPXW231116C04405000,2023-11-15 3:27PM EST,4405.0,101.47,93.6,96.1,0.0,-,40,0,0.00%
2,SPXW231116C04410000,2023-11-15 3:49PM EST,4410.0,97.08,91.0,92.4,0.0,-,125,0,0.00%
3,SPXW231116C04415000,2023-11-15 3:48PM EST,4415.0,92.66,86.0,87.4,0.0,-,127,0,0.00%
4,SPXW231116C04420000,2023-11-15 3:59PM EST,4420.0,85.48,80.5,81.7,0.0,-,205,0,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
311,SPXW231205C04480000,2023-11-15 3:58PM EST,4480.0,63.72,60.8,61.4,0.0,-,3,0,11.71%
312,SPXW231205C04490000,2023-11-14 4:07PM EST,4490.0,56.97,55.2,55.8,0.0,-,16,0,11.69%
313,SPXW231205C04500000,2023-11-15 3:58PM EST,4500.0,51.21,49.2,49.7,0.0,-,845,0,11.47%
314,SPXW231205C04525000,2023-11-15 11:09AM EST,4525.0,46.40,36.1,36.7,0.0,-,3,0,11.13%


In [15]:
filtered_calls_chain['Date'] = filtered_calls_chain['Contract Name'].apply(
    lambda x: x[3:9] if len(x) == 18 else (x[4:10] if len(x) == 19 else x))

In [16]:
filtered_calls_chain['Date'] = pd.to_datetime(
    filtered_calls_chain['Date'], format='%y%m%d')

In [17]:
filtered_calls_chain.set_index("Date", inplace=True)

In [18]:
filtered_calls_chain

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
Date,,,,,,,,,,,
2023-11-16,SPXW231116C04400000,2023-11-15 4:13PM EST,4400.0,99.46,100.8,102.0,0.0,-,100,0,0.00%
2023-11-16,SPXW231116C04405000,2023-11-15 3:27PM EST,4405.0,101.47,93.6,96.1,0.0,-,40,0,0.00%
2023-11-16,SPXW231116C04410000,2023-11-15 3:49PM EST,4410.0,97.08,91.0,92.4,0.0,-,125,0,0.00%
2023-11-16,SPXW231116C04415000,2023-11-15 3:48PM EST,4415.0,92.66,86.0,87.4,0.0,-,127,0,0.00%
2023-11-16,SPXW231116C04420000,2023-11-15 3:59PM EST,4420.0,85.48,80.5,81.7,0.0,-,205,0,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-05,SPXW231205C04480000,2023-11-15 3:58PM EST,4480.0,63.72,60.8,61.4,0.0,-,3,0,11.71%
2023-12-05,SPXW231205C04490000,2023-11-14 4:07PM EST,4490.0,56.97,55.2,55.8,0.0,-,16,0,11.69%
2023-12-05,SPXW231205C04500000,2023-11-15 3:58PM EST,4500.0,51.21,49.2,49.7,0.0,-,845,0,11.47%


In [19]:
df_calls = filtered_calls_chain[['Strike', 'Last Price', 'Implied Volatility']]

In [20]:
df_calls

,Strike,Last Price,Implied Volatility
Date,,,
2023-11-16,4400.0,99.46,0.00%
2023-11-16,4405.0,101.47,0.00%
2023-11-16,4410.0,97.08,0.00%
2023-11-16,4415.0,92.66,0.00%
2023-11-16,4420.0,85.48,0.00%
...,...,...,...
2023-12-05,4480.0,63.72,11.71%
2023-12-05,4490.0,56.97,11.69%
2023-12-05,4500.0,51.21,11.47%


In [21]:
# Pivot the dataframe to get the strike prices as columns and the dates as rows
df_calls_pivot = df_calls.pivot(columns="Strike", values="Last Price")

In [22]:
df_calls_pivot

Strike,4400.0,4405.0,4410.0,4415.0,4420.0,4425.0,4430.0,4435.0,4440.0,4445.0,...,4505.0,4510.0,4515.0,4520.0,4525.0,4530.0,4535.0,4540.0,4545.0,4550.0
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-16,99.46,101.47,97.08,92.66,85.48,76.99,74.46,70.63,66.01,63.96,...,NaN,6.60,NaN,3.70,2.50,1.90,NaN,NaN,NaN,0.45
2023-11-17,100.61,95.30,100.14,92.50,86.45,78.26,71.00,74.11,63.23,65.75,...,15.70,13.80,11.50,6.60,9.30,7.90,3.3,5.20,1.85,1.50
2023-11-20,100.93,101.36,99.57,95.36,85.14,84.00,76.45,79.49,65.53,59.57,...,NaN,19.20,NaN,15.10,11.57,9.04,NaN,NaN,NaN,6.10
2023-11-21,116.54,108.31,113.87,92.20,96.65,94.35,86.90,72.40,78.40,76.79,...,NaN,19.95,NaN,17.00,16.10,15.88,NaN,NaN,NaN,6.20
2023-11-22,109.90,104.43,111.13,105.66,97.37,93.53,88.70,84.80,74.80,73.90,...,NaN,24.40,NaN,22.20,17.30,15.40,NaN,NaN,NaN,9.30
2023-11-24,113.30,105.48,114.98,112.41,92.41,88.94,84.06,86.66,75.91,71.85,...,NaN,26.05,NaN,23.00,20.30,18.00,NaN,14.97,NaN,11.56
2023-11-27,116.66,104.35,105.60,102.80,101.20,103.60,91.58,94.38,86.83,76.34,...,NaN,28.90,NaN,NaN,25.17,NaN,NaN,NaN,NaN,13.77
2023-11-28,122.43,112.35,123.28,107.33,89.19,100.01,84.83,80.74,85.00,58.60,...,NaN,35.18,NaN,NaN,30.48,NaN,NaN,NaN,NaN,17.13
2023-11-29,123.50,104.85,113.99,110.86,105.75,87.45,93.22,92.36,82.10,78.90,...,NaN,40.21,NaN,NaN,31.17,NaN,NaN,NaN,NaN,18.06


In [23]:
# if NaN is present, remove the column
df_calls_pivot.dropna(axis=1, inplace=True)

In [24]:
df_calls_pivot

Strike,4400.0,4405.0,4410.0,4415.0,4420.0,4425.0,4430.0,4435.0,4440.0,4450.0,4460.0,4475.0,4480.0,4490.0,4500.0,4525.0,4550.0
Date,,,,,,,,,,,,,,,,,
2023-11-16,99.46,101.47,97.08,92.66,85.48,76.99,74.46,70.63,66.01,51.30,43.70,24.85,21.10,18.59,11.00,2.50,0.45
2023-11-17,100.61,95.30,100.14,92.50,86.45,78.26,71.00,74.11,63.23,53.88,53.70,33.60,26.90,24.20,18.00,9.30,1.50
2023-11-20,100.93,101.36,99.57,95.36,85.14,84.00,76.45,79.49,65.53,56.58,50.50,39.60,33.20,29.40,21.90,11.57,6.10
2023-11-21,116.54,108.31,113.87,92.20,96.65,94.35,86.90,72.40,78.40,67.38,53.41,48.45,39.50,37.71,22.90,16.10,6.20
2023-11-22,109.90,104.43,111.13,105.66,97.37,93.53,88.70,84.80,74.80,65.72,62.44,52.00,40.70,39.40,28.00,17.30,9.30
2023-11-24,113.30,105.48,114.98,112.41,92.41,88.94,84.06,86.66,75.91,70.36,59.32,54.01,43.62,36.16,31.64,20.30,11.56
2023-11-27,116.66,104.35,105.60,102.80,101.20,103.60,91.58,94.38,86.83,68.66,68.72,56.11,49.63,43.03,37.00,25.17,13.77
2023-11-28,122.43,112.35,123.28,107.33,89.19,100.01,84.83,80.74,85.00,77.80,50.10,54.60,53.05,50.89,38.50,30.48,17.13
2023-11-29,123.50,104.85,113.99,110.86,105.75,87.45,93.22,92.36,82.10,78.97,73.01,58.38,46.00,50.61,39.20,31.17,18.06


In [25]:
# drop first row
df_calls_pivot.drop(df_calls_pivot.index[0], inplace=True)

In [26]:
# df_calls_pivot.drop(df_calls_pivot.index[0], inplace=True)

In [27]:
import datetime as dt
# Calculate the time to expiration in years for each date and set time to expiration as index
df_calls_pivot['Time to Expiration'] = (
    df_calls_pivot.index - dt.datetime.now()).days / 365.25  # type: ignore
df_calls_pivot.set_index('Time to Expiration', inplace=True)

In [28]:
df_calls_pivot

Strike,4400.0,4405.0,4410.0,4415.0,4420.0,4425.0,4430.0,4435.0,4440.0,4450.0,4460.0,4475.0,4480.0,4490.0,4500.0,4525.0,4550.0
Time to Expiration,,,,,,,,,,,,,,,,,
0.000000,100.61,95.30,100.14,92.50,86.45,78.26,71.00,74.11,63.23,53.88,53.70,33.60,26.90,24.20,18.00,9.30,1.50
0.008219,100.93,101.36,99.57,95.36,85.14,84.00,76.45,79.49,65.53,56.58,50.50,39.60,33.20,29.40,21.90,11.57,6.10
0.010959,116.54,108.31,113.87,92.20,96.65,94.35,86.90,72.40,78.40,67.38,53.41,48.45,39.50,37.71,22.90,16.10,6.20
0.013699,109.90,104.43,111.13,105.66,97.37,93.53,88.70,84.80,74.80,65.72,62.44,52.00,40.70,39.40,28.00,17.30,9.30
0.019178,113.30,105.48,114.98,112.41,92.41,88.94,84.06,86.66,75.91,70.36,59.32,54.01,43.62,36.16,31.64,20.30,11.56
0.027397,116.66,104.35,105.60,102.80,101.20,103.60,91.58,94.38,86.83,68.66,68.72,56.11,49.63,43.03,37.00,25.17,13.77
0.030137,122.43,112.35,123.28,107.33,89.19,100.01,84.83,80.74,85.00,77.80,50.10,54.60,53.05,50.89,38.50,30.48,17.13
0.032877,123.50,104.85,113.99,110.86,105.75,87.45,93.22,92.36,82.10,78.97,73.01,58.38,46.00,50.61,39.20,31.17,18.06
0.035616,119.21,114.10,109.86,112.79,105.00,104.10,92.11,85.50,87.04,79.19,77.78,59.27,55.92,54.90,42.90,27.48,20.00


In [29]:
# Convert our vol surface to dataframe for each option price with parameters
volSurfaceLong = df_calls_pivot.melt(ignore_index=False).reset_index()
volSurfaceLong.columns = ["maturity", "strike", "price"]

In [30]:
volSurfaceLong

,maturity,strike,price
0,0.000000,4400.0,100.61
1,0.008219,4400.0,100.93
2,0.010959,4400.0,116.54
3,0.013699,4400.0,109.90
4,0.019178,4400.0,113.30
...,...,...,...
199,0.032877,4550.0,18.06
200,0.035616,4550.0,20.00
201,0.038356,4550.0,21.50
202,0.046575,4550.0,23.50


In [31]:
yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([5.52, 5.54, 5.53, 5.41, 5.27, 4.9, 4.68, 4.52, 4.56, 4.53, 4.87, 4.68]).astype(float)/100

In [32]:
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

In [33]:
# NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities, yeilds)

curve_fit

NelsonSiegelSvenssonCurve(beta0=0.04982862243152366, beta1=0.006715571191473596, beta2=-0.014602722209638338, beta3=-0.00888301212085435, tau1=2.0, tau2=5.0)

In [34]:
# Calculate the risk free rate for each maturity using the fitted yield curve
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(
    curve_fit)  # type: ignore

In [35]:
volSurfaceLong

,maturity,strike,price,rate
0,0.000000,4400.0,100.61,0.056544
1,0.008219,4400.0,100.93,0.056493
2,0.010959,4400.0,116.54,0.056476
3,0.013699,4400.0,109.90,0.056459
4,0.019178,4400.0,113.30,0.056426
...,...,...,...,...
199,0.032877,4550.0,18.06,0.056342
200,0.035616,4550.0,20.00,0.056325
201,0.038356,4550.0,21.50,0.056308
202,0.046575,4550.0,23.50,0.056258


In [36]:
# Define variables to be used in optimization
S0 = 4502.88
r = volSurfaceLong['rate'].to_numpy('float')
K = volSurfaceLong['strike'].to_numpy('float')
tau = volSurfaceLong['maturity'].to_numpy('float')
P = volSurfaceLong['price'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [1e-3,0.1]},
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.03, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,1]},
          "lambd": {"x0": 0.03, "lbub": [-1,1]},
          }

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

In [37]:
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )

    return err

In [38]:
from scipy.optimize import minimize

result = minimize(SqErr, x0, tol=1e-3, method='SLSQP',
                  options={'maxiter': 1000}, bounds=bnds)

In [39]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 3875.7728613907307
       x: [ 2.937e-02  2.001e-03  1.000e-03  1.000e+00 -1.000e+00
           -1.000e+00]
     nit: 8
     jac: [ 6.320e+01  1.027e+00  1.900e-01 -6.262e+02  2.511e+02
            8.369e-01]
    nfev: 57
    njev: 7

In [40]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

(0.029365256922196572,
 0.0020010510982477964,
 0.0010000012272437645,
 0.9999999301907746,
 -0.9999999718774628,
 -0.9999999999310175)

In [41]:
heston_prices = heston_price_rec(
    S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [42]:
volSurfaceLong["heston_price"] = heston_prices[0]

In [43]:
volSurfaceLong['abs_diff'] = abs(
    volSurfaceLong['price'] - volSurfaceLong['heston_price'])

In [44]:
volSurfaceLong

,maturity,strike,price,rate,heston_price,abs_diff
0,0.000000,4400.0,100.61,0.056544,107.949514,7.339514
1,0.008219,4400.0,100.93,0.056493,113.365732,12.435732
2,0.010959,4400.0,116.54,0.056476,116.286412,0.253588
3,0.013699,4400.0,109.90,0.056459,119.206123,9.306123
4,0.019178,4400.0,113.30,0.056426,124.834109,11.534109
...,...,...,...,...,...,...
199,0.032877,4550.0,18.06,0.056342,30.888168,12.828168
200,0.035616,4550.0,20.00,0.056325,33.333473,13.333473
201,0.038356,4550.0,21.50,0.056308,35.725258,14.225258
202,0.046575,4550.0,23.50,0.056258,42.613677,19.113677


In [45]:
# average absolute difference
volSurfaceLong["abs_diff"].mean()

16.06602827300661

In [46]:
import plotly.express as px
import plotly.graph_objects as go

# plot absolute difference between market and heston prices for each strike and maturity
fig = px.scatter_3d(volSurfaceLong, x='strike',
                    y='maturity', z='abs_diff', color='abs_diff')
fig.update_layout(scene=dict(
    xaxis_title='Strike',
    yaxis_title='Maturity',
    zaxis_title='Absolute Difference'),
    width=700,
    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

In [47]:
# Compare with Black Scholes